## **Nugen Intelligence**
<img src="https://nugen.in/logo.png" alt="Nugen Logo" width="200"/>

Domain-aligned foundational models at industry leading speeds and zero-data retention!

### **Using Reasoning for Routine Generation with Nugen API**
 
This code demonstrates how to transform help center articles into executable routines using the Nugen API. By leveraging Nugen's powerful capabilities, we can convert externally-facing help center content into internally-facing routines that can be seamlessly integrated with a large language model (LLM).

The goal is to enhance customer service operations by enabling the LLM to effectively handle customer inquiries and support tasks, ensuring a more efficient and responsive service experience. With Nugen’s advanced API, you can streamline your customer service processes and empower your team with actionable insights from existing help center content.

**Importing Necessary Libraries**


In [1]:
import requests
from IPython.display import display, HTML
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import csv

* requests: A library to make HTTP
* requests to the Nugen API.
* pandas: A library to handle data manipulation and analysis.
* ThreadPoolExecutor: A tool to run multiple tasks concurrently, speeding up processing.
* csv: A module to read and write CSV files.

**Setting Up the Nugen API**

You can access Nugen API key from [here](https://docs.nugen.in/) for FREE! 

In [19]:
url_api_server = "https://api.nugen.in"
api_key =  "your api key" # Replace with your actual Nugen API key
MODEL = "nugen-flash-instruct"

In [20]:
class NugenAPIClient:
    def __init__(self, base_url, api_key):
        self.base_url = base_url
        self.api_key = api_key

    def chat_completions_create(self, model, messages, max_tokens=400, temperature=1):
        url = f"{self.base_url}/inference/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        prompt = "\n".join([message["content"] for message in messages])

        payload = {
            "model": model,
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": temperature
        }

        response = requests.post(url, json=payload, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error {response.status_code}: {response.text}")

# Instantiate Nugen client
client = NugenAPIClient(base_url=url_api_server, api_key=api_key)


1. url: The endpoint for the Nugen API where we send our requests.

2. headers: Includes the authorization token for API access and specifies that we're sending JSON data.

**Defining the Routine Generation Function**

In [21]:
CONVERSION_PROMPT = """
You are a helpful assistant tasked with taking an external facing help center article and converting it into an internal-facing programmatically executable routine optimized for an LLM.
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the customer service policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format.
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., `call the check_delivery_date function`).
      - **If a step requires the customer service agent to take an action** (e.g., process a refund), generate a function call for this action (e.g., `call the process_refund function`).
      - **Define any new functions** by providing a brief description of their purpose and required parameters.
   - **If there is an action an assistant can perform on behalf of the user**, include a function call for this action (e.g., `call the change_email_address function`), and ensure the function is defined with its purpose and required parameters.
      - This action may not be explicitly defined in the help center article, but can be done to help the user resolve their inquiry faster.
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `case_resolution` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.

**Important**: If at any point you are uncertain, respond with "I don't know."

Please convert the customer service policy into the formatted routine, ensuring it is easy to follow and execute programmatically.
"""

# Read articles from CSV
articles = []
with open('../helpcenter_articles.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        articles.append({
            "policy": row["policy"],
            "content": row["content"]
        })

# Function to generate routine using Nugen API
# Debugging step - Print full response to check its structure
def generate_routine(policy):
    try:
        messages = [{
            "role": "user",
            "content": f"{CONVERSION_PROMPT}\n\nPOLICY:\n{policy}"
        }]

        response = client.chat_completions_create(
            model=MODEL,
            messages=messages
        )

        # Print the full response to debug
        print("Full API Response:", response)

        # Modify this according to the actual response structure
        return response['choices'][0].get('text', None)  # Adjust based on actual API response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None



**generate_routine(policy):** This function sends a request to the Nugen API to convert a help center article (policy) into an internal-facing routine.
  1. payload: Contains the request parameters, including the prompt which outlines how to format the routine.
  2. response: Receives the response from the API and extracts the generated routine from it.
  3. Exception handling: Catches any errors that occur during the request and prints an error message.

**Processing Articles Concurrently**


In [22]:
# Function to process each article and generate routine
def process_article(article):
    routine = generate_routine(article['content'])
    return {"policy": article['policy'], "content": article['content'], "routine": routine}

# Execute routine generation concurrently
with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_article, articles))


Full API Response: {'id': 'nugen-1728036394.873584', 'created': 1728036394.873584, 'model': 'nugen-flash-instruct', 'choices': [{'text': "\n\nIf you need help with these steps, please let us know.\n\nWe cannot guarantee the approval of all exemption requests, and it may take several days for us to review and process your application.\n\nIf your organization has any questions or concerns, please contact our Customer Service team.\n\n \n\nCustomer Service Contact Information\nEmail: [insert email]\nPhone: [insert phone number]\nChat: [insert chat widget]\n\nHere is the formatted routine:\n\n1. Review the customer's request for a VAT exemption.\n    a. If the customer is eligible for a tax exemption, then:\n        1. Request the customer to obtain a current tax exemption certificate from their state or local government and/or their fully completed non-profit sales tax exemption forms.\n        2. Instruct the customer to include their organization id, invoice number, or email address ass

1. process_article(article): Calls the generate_routine function for each article and returns a dictionary with the policy, content, and generated routine.

2. Reading Articles: Loads help center articles from a CSV file into the articles list.

3. Concurrent Processing: Uses ThreadPoolExecutor to process multiple articles at once, speeding up the routine generation.

**Storing and Displaying Results**

In [23]:
# Convert results to a DataFrame
df = pd.DataFrame(results)

# Set display options to show all text in the dataframe cells
pd.set_option('display.max_colwidth', None)

# Function to display formatted text in HTML
def display_formatted_dataframe(df):
    def format_text(text):
        return text.replace('\n', '<br>') if text else "No routine generated"

    df_formatted = df.copy()
    df_formatted['content'] = df_formatted['content'].apply(format_text)
    df_formatted['routine'] = df_formatted['routine'].apply(format_text)

    display(HTML(df_formatted.to_html(escape=False, justify='left')))

# Display the dataframe with formatted text
display_formatted_dataframe(df)

,policy,content,routine
0,Delete Payment Method,"How do I delete my payment method?Updated over a week agoWe keep your payment method on file to cover any outstanding charges on your account. To stop charges to your payment method, please follow the steps below.## ChatGPT You can cancel your ChatGPT Plus subscription to stop further charges at any time: Click on 'My Plan' in the ChatGPT sidebar. Click on 'Manage my subscription' in the pop-up window.Select 'Cancel Plan'. Please note that your cancellation will take effect the day after the next billing date, and you can continue using our services until then. To avoid being charged for your next billing period, please cancel your subscription at least 24 hours before your next billing date. ## APIWe'll need to keep a payment method on file to account for any outstanding usage costs. You're welcome to cancel your pay-as-you-go service, by clicking 'Cancel paid account' in your billing overview. After the current month's invoice has been issued, the current card will no longer be charged. If you'd like to continue using the service, add a new payment method in the billing overview page and select 'Set as default'. You'll then be able to delete the old payment method.","If you have any questions or need help with the process, feel free to ask."
1,Business Associate Agreement,"How can I get a Business Associate Agreement (BAA) with OpenAI?Information about HIPAA compliance for healthcare companiesThe Health Insurance Portability and Accountability Act (HIPAA) is a U.S. federal law that requires privacy and security protections for protected health information (PHI). Our API platform can be a great fit for any covered entity or business associate looking to process protected health information, and we’d be happy to assist you in fulfilling your HIPAA compliance. To use our API platform, you’ll first need a BAA with OpenAI.How do I get started?If you require a BAA before you can use our API, email us at baa@openai.com with details about your company and use case.Our team will respond within 1-2 business days. We review each BAA request on a case-by-case basis and may need additional information. The process is usually completed within a few business days.Can I get a BAA for ChatGPT?If you're interested in exploring a BAA for ChatGPT Enterprise, please contact sales. What happens if I’m not approved?We are able to approve most customers that request BAAs, but occasionally a use case doesn’t pass our team's evaluation. In that case, we’ll give feedback and context as to why that is and give you the opportunity to update your intended use of our API and re-apply. Are all API services covered by the BAA?No, only endpoints that are eligible for zero retention are covered by the BAA. You can see a list of those endpoints. Is an enterprise agreement requirement to sign a BAA?No, an enterprise agreement is not required to sign a BAA.","How can I get an enterprise agreement?If you are interested in purchasing an enterprise agreement, please contact our sales team at sales@openai.com. Please find a copy of the BAA on our website.1. **Review the customer's request**: - If the customer is looking for a BAA for OpenAI, then proceed to the next step. - If the customer is looking for a BAA for ChatGPT Enterprise, then direct them to contact sales. - If the customer is looking for information about HIPAA compliance, then provide information about HIPAA compliance for healthcare companies.2. **Determine the customer's use case**: - If the customer has already provided details about their company and use case, then proceed to the next step. - If the customer has not provided details about their company and use case, then ask for additional information. - `call the check_use_case function` to verify the customer's use case.3. **Process the BAA request**: - If the customer's use case is approved, then proceed to the next step. - If the customer's use case is not approved, then provide feedback and cont

1. Creating a DataFrame: Converts the results into a Pandas DataFrame for easier manipulation and viewing.
2. Displaying the Data: Defines a function display_formatted_dataframe(df) to format and display the DataFrame in a readable HTML format.

**Summary**

This code converts help center articles into internal-facing routines using the Nugen API. It processes multiple articles concurrently to improve efficiency and displays the results in an easily readable format. By following the steps outlined in this documentation, you can adapt and use the code for your own routine generation needs.